# Семинар 0. Вспомнить всё!

План семинара:

- [ ] Вспоминаем основные понятия из статистики: модели и оценки параметров
- [ ] Вспоминаем методы оценок параметров. Основной -- ML.
- [ ] Задача линейной регрессии
- [ ] Вспоминаем тестирование гипотез

### Основные понятия из статистики

Так как данный курс посвящён временным рядам, следует вообще говоря определить, что это такое. В реальной жизни мы наблюдаем много явлений, протяжённых во времени. Будь то котировки акций, уровень ВВП или количество продаж нового айфона. Притом критически важным является деталь, что мы наблюдаем не случайный процесс, а его некоторую **реализацию**. Это и является временным рядом. Мы наблюдаем некоторый частный случай того, как могли произойти события. Но хотелось бы, конечно, видеть полную картину.

Теперь нашей задачей является восстановление общего закона протекания процесса в зависимости от реализации. Но можем ли мы в принципе быть уверены, что сможем его восстановить? В какой-то мере ответ на это даёт [Теорема Дуба о разложении ~~на плесень и липовый мёд~~](https://wikichi.ru/wiki/Doob_decomposition_theorem). Говоря в очень упрощённых терминах, она говорит о том, что почти любой "хороший" процесс можно разложить на прогнозируемую (детерминированную) и принципиально непрогнозируемую части. Следственно, мы никогда не сможем восстановить процесс идеально. Но тем не менее, часть процессов более склонна к детерминированному поведению, а часть -- менее. Например дневная температура яввляется очень сезонной величиной, то есть имеет паттерн, удобный для прогнозирования. С другой стороны, котировки акций наиболее близки к хаотичному движению и весьма трудно поддаются прогнозированию. Этим занимаются скорее в области количественных финансов. В нашем курсе котировки акций могут быть рассмотрены скорее из-за удобства и качества данных, но не более чем для иллюстрации. Исключение составит семинар про GARCH-модели. Также всюду будем рассматривать только дискретные временные ряды.

Удобным представлением для описания дискретных процессов являются случайные величины. То есть мы предполагаем, для каждой точки времени существует некоторая случайная величина, породившая наблюдение. Следовательно, у каждой такой величины существует некоторое **распределение**. Будем предполагать, что это распределение задаётся параметрической функцией. Её параметры могут меняться с течением времени, но функциональная форма -- нет.  Далее нашей задачей будет подбор таких параметров распределения, которые бы максимально приблизили порождаемый процесс к наблюдаемому временному ряду. Для того, чтобы решать такие задачи, необходимо вспомнить ряд определений и зафиксировать обозначения.

$X - \text{некоторая случайная величина, которую мы хотим оценить}$

$X_1 \cdots X_n - \text{n наблюдений (выборка) о случайной величине, тоже случайные величины}$ 

$x_1 \cdots x_n - \text{реализация выборки, конкретные неслучайные числа}$

$\text{Предполагаем, что }X_i \sim f(\theta), \text{ где } \theta - \text{ некоторый набор параметров }$

Наша цель -- получить такую $\hat{\theta} = g(X_1, \cdots, X_n)$, которая максимально подгонит распределение под данные.




In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import plotly.graph_objects as go

np.set_printoptions(precision=3)



In [2]:
sample = np.random.normal(1, 2, 50)

mu_hat = np.mean(sample)
sigma_hat = np.std(sample)

est_x = np.linspace(-10, 10)
est_y = ss.norm.pdf(est_x, mu_hat, sigma_hat)

fig = go.Figure()

fig.add_trace(go.Histogram(x=sample, nbinsx=20, histnorm="probability density", name="Нормированные частоты"))
fig.add_trace(go.Scatter(x=est_x, y=est_y, name="Оценённая плотность"))

fig.update_layout(
    title={
        'text': "Оценка плотности распределения",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    legend={
        'x':0.7,
        'y':1},
    autosize=False,
    width=800,
    height=400)
fig.show()

## Методы оценки параметров

1. Метод моментов
    * Идея: Истинные моменты должны равняться выборочным. Если моменты распределений похожи, то и сами они схожи.
    * Необходимо выписать систему моментных тождеств. Моментное тождество - это уравнение, в котором с одной стороны стоит выборочный момент, а с другой -- истинный. Решив эти уравнения относительно параметров, получим оценки. Подобно рассматривать в курсе не будем
2. Метод максимального правдоподобия. 
    * Идея: Плотность - это ненормированная вероятность. Давайте максимизируем совместную вероятность (совместную плотность) получить нашу выборку.
    
    Плотность выборки -- это совместная плотность всех n наблюдений. Её можно расписать следующим образом:
    $$L(X|\theta) = f(X_n, \dots, X_1|\theta) = f(X_{n} | X_{n-1}, \dots, X_1,\theta)  \cdot f( X_{n-1}, \dots, X_1|\theta) = $$

    $$f(X_{n} | X_{n-1}, \dots, X_1,\theta)  \cdot f(X_{n-1} | X_{n-2}, \dots, X_1,\theta)  \cdot f( X_{n-2}, \dots, X_1|\theta) = $$

    $$ f(X_{n} | X_{n-1}, \dots, X_1|\theta)\cdot \dots \cdot f( X_{2} | X_1, \theta) \cdot f(X_1|\theta) = \prod_{i=1}^n f(X_i | \{X_j\}_{j=1}^{i-1}, \theta) \to \max_{\theta}$$


    * Предположим для начала независимость наблюдений в выборке. Получим произведение независимых вероятностей. Полученную функцию, а точнее её логарифм, максимизируем по параметрам.

$$L(X|\theta) = \prod_{i=1}^n f(X_i|\theta) \to \max_{\theta}$$

$$l(X|\theta) = \sum_{i=1}^n ln(f(X_i|\theta)) \to \max_{\theta}$$





In [3]:
from scipy.stats import norm
from scipy.optimize import minimize

np.set_printoptions(precision=2)
np.random.seed(1)
# Пусть мы имеем выборку из нормального распределения. 
n_sample = 500
X = np.random.normal(loc=5, scale=3, size =n_sample)

# Подсчёт лог-правдоподобия
def neg_loglik(params):
    loc, scale = params
    return -np.sum(np.log(norm.pdf(X, loc, scale)))

# Градиент лог-правдоподобия
def neg_loglik_grad(params):
    loc, scale = params 

    loc_grad = -np.sum(X - loc) / (scale **2)
    scale_grad = X.shape[0]/scale - np.sum((X - loc)**2) / scale**3

    return loc_grad, scale_grad

    
params_0 = [3,7]
res = minimize(neg_loglik, params_0, jac=neg_loglik_grad, method="BFGS")

print(f'Логарифм правдоподобия в оптимуме: {neg_loglik([5, 3])}')
print(f'Логарифм правдоподобия в оценке: {neg_loglik(res.x)}')

Логарифм правдоподобия в оптимуме: 1253.7270081078705
Логарифм правдоподобия в оценке: 1252.9456865924642


In [4]:
res.x

array([5.16, 2.97])

Однако во временных рядах существует ряд проблем. Например, наблюдения не являются независимыми между собой. Каждое последующее наблюдение может зависеть от предыдущих, но, что логично не зависит от будущих. В таком случае правдоподобие не упрощается, и у нас остаётся необходимость выписывать условные вероятности в правдоподобии. Притом, последние наблюдения будут зависеть от всех предыдущих. Возвращаемся к исходной формуле:

 $$\prod_{i=1}^n f(X_i | \{X_j\}_{j=1}^{i-1}, \theta) \to \max_{\theta}$$

Чтобы хоть как-то упростить задачу, исследователи обычно вносят предпосылку о наличии марковского свойства для фиксированного горизонта $k$: 

$$\prod_{i=1}^n f(X_i | \{X_j\}_{i-1-k}^{i-1}, \theta) \to \max_{\theta}$$

Иными словами, каждый элемент нашего правдоподобия начинает зависеть не от всей предыстории, а от $k$ последних элементов. Это может существенно упростить запись правдоподобия. Теперь нам необходимо выписать вместо $n$ различных плотностей только $k+1$ различных плотностей. Чисто математически это существенно упрощает анализ правдоподобия.

## Свойства оценок параметров

Важно отметить, что в общей постановке оценки являются функциями от выборки, которая, в свою очередь, является случайной величиной. Следовательно, оценки тоже являются случайными величинами.

1. Оценка $\hat{\theta}$ параметра $\theta$ является несмещённой, если $\mathbb{E}(\hat{\theta}) = \theta$
2. Оценка $\hat{\theta}$ параметра $\theta$ является состоятельной, если $plim(\hat{\theta}) = \theta$ при $n \to \infty$
3. Оценка $\hat{\theta}_1$ более эффективна чем  $\hat{\theta}_2$, если $\mathbb{E}(\hat{\theta}_1 - \theta)^2 < \mathbb{E}(\hat{\theta}_2 - \theta)^2$

## Задача линейной регрессии

Задача регрессии - одна из ключевых в статистике. Мы предполагаем некоторую зависимость между переменными и пытаемся её оценить. Формализуем:

Предполагаем зависимость следующего вида: $Y = f(X, \theta) + \varepsilon$, где $\varepsilon$ (случайная величина) - некоторый шум и невключённые в модель факторы. В самой стандартной постановке в качестве $f$ выбирают линейную функцию.

$$ Y = \beta_0 + \beta_1 X + \varepsilon $$

Для того чтобы оценить беты, необходимо снова прибегнуть к какому-либо методу

+ МНК. Метод наименьших квадратов предлагает просто минимизировать $\sum_{i=1}^n(Y - X\beta)^2$. Эта задача имеет аналитическое решение:.
  Плюсы: 
    1. Есть аналитическое решение.  $\hat{\beta} = (X'X)^{-1}X'Y$.
    2. Выпуклый функционал, что позволяет эффективно решать задачу численно
    3. При выполнении предпосылок теоремы Гаусса-Маркова, оценки МНК являются BLUE (Best Linear Unbiased estimators), то есть линейными по Y, несмещёнными и эффективными (а часто и состоятельными, но это уже зависит от конкретной функциональной формы). 

  Минусы:
  
    1. Плохо применим в контексте временных рядов, так как одна из предпосылок ТГМ -- независимость наблюдений.
    2. Дорого обращать матрицу для получения аналитического решения.
    3. В отличие от ML-оценок, оценки МНК не инвариантны относительно гладких преобразований: $\widehat{g(\theta)} \neq g(\hat{\theta})$

+ Можно указать явное распределение для $\varepsilon$, выразить эту величину из уравнения, а далее использовать стандартные методы для распределений - метод моментов, максимального правдоподобия, etc. Обычно используют метод максимального правдоподобия.

  Плюсы: 

    1. Гибкость при выборе распределений. Можно взять любое, которое будет осмысленным и более-менее легко задаваться и оптимизироваться.
    2. Инвариантность ML-оценок параметров $\widehat{g(\theta)} = g(\hat{\theta})$
    3. Возможность учитывать в правдоподобии условные плотности
    4. При достаточно базовых предпосылках могут присутствовать хорошие свойства оценок (например, несмещённость или эффективность при обращении неравенства Крамера-Рао в равенство)

  Минусы:
  
    1. Не для всех распределений правдоподобие легко выписать (особенно при наличии условных плотностей)
    2. Достаточно мало распределений позволяют получить аналитическое решение. В функционале часто присутствует много локальных минимумов и его может быть сложно оптимизировать

## Тестирование гипотез

Предположим теперь, что мы смогли подобрать хороший метод и оценить нашу случайную величину. Мы получили некоторые оценки $\hat{\theta}$ для параметров $\theta$. После этого нам бы по-хорошему хотелось бы как-то использовать это знание и сделать выводы о рассматриваемой величине. Для простоты рассмотрим следующий пример.

Пусть мы имеем некоторую реализацию процесса $y$. Выглядит она следующим образом:





In [5]:
n_samples = 300

x = np.arange(n_samples)
y = np.random.normal(5, 3, 300)

fig = go.Figure() 

fig.add_trace(go.Scatter(x=x, y=y, name="Симулированный процесс"))

fig.update_layout(
    title={
        'text': "Траектория реализации процесса y_t",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis={'title_text':'Время'},
    yaxis={'title_text':'Значение'},
    legend={
        'x':0.7,
        'y':1},
    autosize=False,
    width=800,
    height=400)
fig.show()

  Мы сгенерировали процесс вида $y_t = 5 + 3\varepsilon_t, \quad
  \varepsilon_t \sim \mathbb{N}(0,1),\quad \varepsilon_t - iid$ По сути, это процесс белого шума, сдвинутый на константу. Подробнее о процессах белого шума мы будем говорить чуть позже в курсе. 
  
  Теперь предположим, что мы забыли процесс генерации данных и видим лишь результат. Попытаемся оценить характеристики этого процесса. Мы не знаем, как в точности распределён $y_t$, но можем произвести оценку. Глядя на график, не наблюдается какой-то чёткой зависимости между наблюдениями. Попробуем применить метод максмиального правдоподобия для случая независимых величин и используя нормальное распределение. В результате получаем оценку $\widehat{y_t} \sim \mathbb{N}(\hat{\mu}, \widehat{\sigma^2})$. Далее нам бы хотелось проверить полученные результаты на наличие статистических закономерностей. Например, правда ли, что $\mu = 5$. Если мы сможем статистически проверить такую гипотезу, то значит сможем получить некоторые определённые данные о нашем процессе и сможем упростить модель, заменив в оценке $\hat{\mu}$ на 5.
  
    

  В качестве алгоритма по проверке гипотез можно использовать следующие шаги:
  1. Обычно при тестировании вам даны нулевая гипотеза $\left(H_0\right)$ и альтернативная гипотеза $\left(H_1 \text{ или } H_A\right)$ Например:

      $$H_0: \mu = 5$$
      
      $$H_1: \mu \neq 5$$

      Гипотезы бывают простые и сложные. Простые подразумевают под собой одно распределение, а сложные - семейство распределений. В нашем примере нулевая гипотеза простая, а альтернативная - сложная.
  
  2. Далее исследователь выбирает уровень значимости. Выбрать уровень значимости, с которым мы будем проверять гипотезу. Обычно берут 10%, 5% или 1%. В тестировании гипотез это называется ошибкой первого рода. То есть с какой вероятностью нулевая гипотеза будет отвергнута при условии, что она верна.

<center>
<table style="width:30%">
<tr>
<th></th>
<th>H_0 не отвергается</th>
<th>H_0 отвергается</th>
</tr>

<tr>
<td>H_0 верна</td>
<td>-</td>
<td>Ошибка I рода</td>
</tr>

<tr>
<td>H_0 не верна</td>
<td>Ошибка II рода</td>
<td>-</td>
</tr>
</table>
</center>
  3. Для тестирования гипотезы нам необходима специальная тестовая статистика. Это некоторая величина, распределение которой при выполнении H0 мы знаем. Тестовую статистику вычисляют на основе данных. Если найденное значение статистики экстремально и вероятность получения таких (или более экстремальных) данных меньше, чем уровень значимости, то гипотеза отвергается. Если больше, то гипотеза не отвергается.
  
  ![algorithm](images/sem_1_pvalue.jpg)